In [2]:
import statsmodels.api as sm
import scipy.linalg
import numpy as np
import scipy
from persuasion import get_persuasion_data

"""
Notebook to calculate inter-human variance of agreement changes (persuasiveness-metrics).
"""

df = get_persuasion_data()
for key in ["Unnamed: 0", "claim", "prompt_type"]:
    del df[key]

Index(['worker_id', 'argument', 'source', 'rating_initial', 'rating_final',
       'persuasiveness_metric'],
      dtype='object')

In [6]:
grouped = df.groupby(['argument', 'rating_initial'])

distances = []
pairs = []
for _, group in grouped:
    if len(group) > 1:
        group = group.sort_index()  # Ensure rows are in the original order
        for i in range(len(group)-1):
            pairs.append((group.iloc[i], group.iloc[i+1]))
            distances.append(abs(group.iloc[i]['persuasiveness_metric'] - group.iloc[i+1]['persuasiveness_metric']))

print(f"Accuracy: {len([x for x in distances if x == 0]) / len(distances)}")
print(f"Average Distance Score: {np.linalg.norm([1-x/7 for x in distances], 1) / len(distances)}")
print(f"Mean Squared Distance: {np.linalg.norm(distances, 2) / (len(distances)**0.5)}")

x_values = []
y_values = []

for first_row, second_row in pairs:
    x_values.append(first_row['persuasiveness_metric'])
    y_values.append(second_row['persuasiveness_metric'])


# Regression
if x_values and y_values:
    X = sm.add_constant(x_values) 
    y = y_values
    model = sm.OLS(y, X).fit()
    print(model.summary())
    res = scipy.stats.linregress(x_values, y_values)
    print(res)
else:
    print("No pairs found for regression analysis.")


Accuracy: 0.6465408805031446
Average Distance Score: 0.9279424977538185
Mean Squared Distance: 0.9562768473643141
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.046
Model:                            OLS   Adj. R-squared:                  0.044
Method:                 Least Squares   F-statistic:                     37.88
Date:                Sun, 04 Aug 2024   Prob (F-statistic):           1.20e-09
Time:                        11:03:56   Log-Likelihood:                -987.00
No. Observations:                 795   AIC:                             1978.
Df Residuals:                     793   BIC:                             1987.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------